In [1]:
import pandas as pd
import sklearn as skl
from sklearn.neighbors import NearestNeighbors
import scipy
from scipy.spatial import distance
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import cKDTree

In [2]:
# Load file
data = pd.read_csv('14_11_10_Green_20to21min.csv')

# Drop unused columns
data = data.drop(['Unnamed: 0','Unnamed: 0.1','source_file'], axis=1)

# Drop conf columns
data = data[data.columns.drop(list(data.filter(regex='conf')))]
data.head(5)


,frame_num,person_id,keypoint1_x,keypoint1_y,keypoint2_x,keypoint2_y,keypoint3_x,keypoint3_y,keypoint4_x,keypoint4_y,...,keypoint21_x,keypoint21_y,keypoint22_x,keypoint22_y,keypoint23_x,keypoint23_y,keypoint24_x,keypoint24_y,keypoint25_x,keypoint25_y
0,36000,-1,1127.7800,721.728,1083.650,780.396,1036.490,789.370,1071.730,883.377,...,0.0,0.0,0.000,0.00,0.000,0.00,0.000,0.00,0.000,0.0
1,36000,-1,44.6567,677.403,0.000,0.000,0.000,0.000,0.000,0.000,...,0.0,0.0,0.000,0.00,0.000,0.00,0.000,0.00,0.000,0.0
2,36000,-1,603.9880,677.432,600.887,712.665,562.695,715.790,600.793,783.424,...,0.0,0.0,0.000,0.00,0.000,0.00,0.000,0.00,0.000,0.0
3,36000,-1,480.2200,762.866,453.769,759.812,406.570,762.721,383.106,848.134,...,0.0,0.0,539.154,1027.72,492.045,1074.82,483.149,1074.84,489.049,1051.3
4,36000,-1,512.6760,644.945,524.307,645.110,497.939,645.168,503.677,677.518,...,0.0,0.0,0.000,0.00,0.000,0.00,0.000,0.00,0.000,0.0


In [4]:
# NEW CODE REWRITTEN Using cKDTree 
# Notes: Need some sort of method of noting relations to know which person_id it belongs to and need way of determining sum

def formatted_for_tree_array(considered_keypoint_coord, frame, keypoint_idx):
    func_array = considered_keypoint_coord
    #print(func_array)
    for person,person_data in data[data.frame_num == frame+1].iterrows():
        person_array = person_data[2:].to_numpy()
        relev_keypoint_from_nextframe = np.split(person_array, 25)[keypoint_idx]
        func_array = np.append(func_array,relev_keypoint_from_nextframe)
    func_array = np.split(func_array,len(func_array)/2)
    func_array = np.array(func_array)
    return func_array

for frame in data.groupby(['frame_num']): # groupby returns tuple (index, dataframe)
    for person,person_data in frame[1].iterrows(): # iterating through rows of the current frame dataframe
        person_array = person_data[2:].to_numpy() # convert to numpy array just the coordinates of keypoints [2:]
        keypoint_idx = 0
        for keypoint_coord in np.split(person_array, 25): # split to create list of coordinates (x,y) as np arrays
            print('Frame {} \nPerson {}\nKeypoint #{}'.format(frame[0], person, keypoint_idx+1))
            array = formatted_for_tree_array(keypoint_coord,frame[0],keypoint_idx) # call function to retrieve relevant next frame coordinates
            tree = cKDTree(array)
            idx = tree.query_ball_point(keypoint_coord, 110, return_sorted = True)
            print('\n Values within 110 pixels of {}: \n {} \n'.format(keypoint_coord, array[idx])) # Gets the indices 
            keypoint_idx += 1
        break
    break
            

Frame 36000 
Person 0
Keypoint #1

 Values within 110 pixels of [1127.78   721.728]: 
 [[1127.78   721.728]
 [1127.73   721.732]
 [1124.8    642.177]
 [1021.98   692.195]] 

Frame 36000 
Person 0
Keypoint #2

 Values within 110 pixels of [1083.65   780.396]: 
 [[1083.65   780.396]
 [1083.63   777.471]] 

Frame 36000 
Person 0
Keypoint #3

 Values within 110 pixels of [1036.49  789.37]: 
 [[1036.49   789.37 ]
 [1033.59   789.296]] 

Frame 36000 
Person 0
Keypoint #4

 Values within 110 pixels of [1071.73   883.377]: 
 [[1071.73   883.377]
 [1069.01   880.644]] 

Frame 36000 
Person 0
Keypoint #5

 Values within 110 pixels of [1145.39   906.919]: 
 [[1145.39   906.919]
 [1145.42   906.885]] 

Frame 36000 
Person 0
Keypoint #6

 Values within 110 pixels of [1136.55   765.695]: 
 [[1136.55   765.695]
 [1136.58   765.655]
 [1145.43   668.601]] 

Frame 36000 
Person 0
Keypoint #7

 Values within 110 pixels of [1151.28   833.529]: 
 [[1151.28   833.529]
 [1148.39   833.518]] 

Frame 36000 
Pe

In [12]:
# OLD CODE STARTS HERE
frame_person_keypoints_dict = {}
for current_frame in data.frame_num.unique():
    frame_person_keypoints_dict[current_frame] = {}
    person_id = 0
    for i,person in data[data.frame_num==current_frame].iterrows():
        starter = 5
        count = 0
        person_25coordinates = np.zeros(shape=(25,2))
        while count<25:
            person_25coordinates[count]=person.iloc[starter:starter+2]
            starter = starter + 3
            count = count + 1
        frame_person_keypoints_dict[current_frame][person_id] = person_25coordinates
        person_id = person_id + 1

In [124]:
# Plot of euclidian distances for all points in data
frame_person_keypoints_dict = {}
for current_frame in data.frame_num.unique():
    frame_person_keypoints_dict[current_frame] = {}
    person_id = 0
    for i,person in data[data.frame_num==current_frame].iterrows():
        starter = 5
        count = 0
        person_25coordinates = []
        while count<25:
            person_25coordinates.append(person.iloc[starter:starter+2])
            starter = starter + 3
            count = count + 1
        frame_person_keypoints_dict[current_frame][person_id] = person_25coordinates
        person_id = person_id + 1

# Rule of thumb can be 1% of the video height
# Rule of thumb can be 1% of the video width
# Height of someone's head and that can be the rule of thumb for effective
# Ambiguity distance

In [139]:
col_names = ['keypoint1']
for i in range(1, 26):
    col_names.extend(['keypoint' + str(i)])
distance_df = pd.DataFrame(columns=col_names)

for frame in data.frame_num.unique(): # To keep increasing order of frames
    for person in frame_person_keypoints_dict[frame].keys():
        person_dists = [] 
        for keypoint_coordinate in frame_person_keypoints_dict[frame][person]:
            for person_next_frame in frame_person_keypoints_dict[frame+1].keys():
                distance = distance.euclidean(frame_person_keypoints_dict[frame][person][keypoint_coordinate], frame_person_keypoints_dict[frame+1][person_next_frame][keypoint_coordinate])
                person_dists.append(distance)
        
dst = distance.euclidean(a, b)

keypoint1_x    1127.78
keypoint1_y    721.728
Name: 0, dtype: object

In [67]:
d1 = [column for column in [data.columns] if 'x|y' in column]

In [74]:
df1 = data.iloc[:,5:]

In [89]:
a = [data.iloc[:,5:7]]
a

[       keypoint1_x  keypoint1_y
 0        1127.7800      721.728
 1          44.6567      677.403
 2         603.9880      677.432
 3         480.2200      762.866
 4         512.6760      644.945
 5           0.0000        0.000
 6         197.8120      701.006
 7        1610.4400      565.594
 8        1266.1800      712.673
 9           0.0000        0.000
 10       1124.8000      642.174
 11          0.0000        0.000
 12        730.3710      536.211
 13        421.3450      644.891
 14        880.5540      645.187
 15        780.5110      662.642
 16       1563.2300      806.847
 17        977.7560      621.413
 18          0.0000        0.000
 19          0.0000        0.000
 20          0.0000        0.000
 21       1021.9600      692.181
 22          0.0000        0.000
 23          0.0000        0.000
 24        789.1400      600.924
 25       1589.7800      718.563
 26        933.5920      621.660
 27         44.6702      677.515
 28        603.9690      677.477
 29       